In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="yAGqrP4bbcFc0kgTqBks")
project = rf.workspace("aarhus-universitet-r1lgs").project("deep-learning-wnuyb")
version = project.version(12)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Deep-Learning-12 in yolov11:: 100%|██████████| 1128/1128 [00:00<00:00, 2943.37it/s]


In [46]:
from pathlib import Path

dataset_path = Path("./Deep-learning-12/train") 
labels = sorted(dataset_path.rglob("*labels/*.txt"))  # all data in 'labels'

In [48]:
import yaml

yaml_file = "./Deep-learning-12/data.yaml"  
with open(yaml_file, "r", encoding="utf8") as y:
    classes = yaml.safe_load(y)["names"]
cls_idx = sorted(classes.keys())

print(cls_idx)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [49]:
import pandas as pd

indx = [label.stem for label in labels]  # uses base filename as ID (no extension)
labels_df = pd.DataFrame([], columns=cls_idx, index=indx)

In [50]:
print(labels_df)

                                                     0    1    2    3    4   \
AEBLER_I_POSE_png.rf.a73452dcbe3095dda703647be2...  NaN  NaN  NaN  NaN  NaN   
AEGTE_MAYONNAISE_png.rf.4c4f2cfa18214e9af5aad1e...  NaN  NaN  NaN  NaN  NaN   
AEble_Belle_de_Boskoop_oko-_1_png.rf.2e9d1f7f78...  NaN  NaN  NaN  NaN  NaN   
AEble_Belle_de_Boskoop_stor_1_png.rf.5a09d80e53...  NaN  NaN  NaN  NaN  NaN   
AEble_Royal_Gala_1_png.rf.3c680fb79e6491fc6b4e4...  NaN  NaN  NaN  NaN  NaN   
...                                                 ...  ...  ...  ...  ...   
images_jpg.rf.aab537fa8ad02a17ae4da3028a9577dc      NaN  NaN  NaN  NaN  NaN   
images_jpg.rf.af5f769c9f1cbf9ae739b038a314bc08      NaN  NaN  NaN  NaN  NaN   
piske_jpg.rf.10c76dabef39c461cb23354afdd2e99d       NaN  NaN  NaN  NaN  NaN   
piske_jpg.rf.cc045bca5130c7e19b85793579cd0dcb       NaN  NaN  NaN  NaN  NaN   
smagstest-af-rugbroed_jpg.rf.4315ba178c969b3f3f...  NaN  NaN  NaN  NaN  NaN   

                                                   

In [51]:
from collections import Counter

for label in labels:
    lbl_counter = Counter()

    with open(label, "r") as lf:
        lines = lf.readlines()

    for line in lines:
        # classes for YOLO label uses integer at first position of each line
        lbl_counter[int(line.split(" ")[0])] += 1

    labels_df.loc[label.stem] = lbl_counter

labels_df = labels_df.fillna(0.0)  # replace `nan` values with `0.0`

/var/folders/lq/v36xrkmd52s39s9cz2xzy7t80000gn/T/ipykernel_10640/4074969495.py:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  labels_df = labels_df.fillna(0.0)  # replace `nan` values with `0.0`


In [52]:
# display non-zero values
print(labels_df)


                                                     0    1    2    3    4   \
AEBLER_I_POSE_png.rf.a73452dcbe3095dda703647be2...  1.0  0.0  0.0  0.0  0.0   
AEGTE_MAYONNAISE_png.rf.4c4f2cfa18214e9af5aad1e...  0.0  0.0  0.0  0.0  0.0   
AEble_Belle_de_Boskoop_oko-_1_png.rf.2e9d1f7f78...  4.0  0.0  0.0  0.0  0.0   
AEble_Belle_de_Boskoop_stor_1_png.rf.5a09d80e53...  1.0  0.0  0.0  0.0  0.0   
AEble_Royal_Gala_1_png.rf.3c680fb79e6491fc6b4e4...  1.0  0.0  0.0  0.0  0.0   
...                                                 ...  ...  ...  ...  ...   
images_jpg.rf.aab537fa8ad02a17ae4da3028a9577dc      0.0  0.0  0.0  0.0  0.0   
images_jpg.rf.af5f769c9f1cbf9ae739b038a314bc08      0.0  0.0  0.0  0.0  0.0   
piske_jpg.rf.10c76dabef39c461cb23354afdd2e99d       0.0  0.0  0.0  0.0  0.0   
piske_jpg.rf.cc045bca5130c7e19b85793579cd0dcb       0.0  0.0  0.0  0.0  0.0   
smagstest-af-rugbroed_jpg.rf.4315ba178c969b3f3f...  0.0  0.0  0.0  0.0  0.0   

                                                   

In [53]:
from sklearn.model_selection import KFold

ksplit = 5
kf = KFold(n_splits=ksplit, shuffle=True, random_state=20)  # setting random_state for repeatable results

kfolds = list(kf.split(labels_df))

In [54]:
folds = [f"split_{n}" for n in range(1, ksplit + 1)]
folds_df = pd.DataFrame(index=indx, columns=folds)

for idx, (train, val) in enumerate(kfolds, start=1):
    folds_df[f"split_{idx}"].loc[labels_df.iloc[train].index] = "train"
    folds_df[f"split_{idx}"].loc[labels_df.iloc[val].index] = "val"

/var/folders/lq/v36xrkmd52s39s9cz2xzy7t80000gn/T/ipykernel_10640/1666497045.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  folds_df[f"split_{idx}"].loc[labels_df.iloc[train].index] = "train"
/var/folders/lq/v36xrkmd52s39s9cz2xzy7t80000g

In [55]:
fold_lbl_distrb = pd.DataFrame(index=folds, columns=cls_idx)

for n, (train_indices, val_indices) in enumerate(kfolds, start=1):
    train_totals = labels_df.iloc[train_indices].sum()
    val_totals = labels_df.iloc[val_indices].sum()

    # To avoid division by zero, we add a small value (1E-7) to the denominator
    ratio = val_totals / (train_totals + 1e-7)
    fold_lbl_distrb.loc[f"split_{n}"] = ratio

In [56]:
import datetime

supported_extensions = [".jpg", ".jpeg", ".png"]

# Initialize an empty list to store image file paths
images = []

# Loop through supported extensions and gather image files
for ext in supported_extensions:
    images.extend(sorted((dataset_path / "images").rglob(f"*{ext}")))

# Create the necessary directories and dataset YAML files (unchanged)
save_path = Path(dataset_path / f"{datetime.date.today().isoformat()}_{ksplit}-Fold_Cross-val")
save_path.mkdir(parents=True, exist_ok=True)
ds_yamls = []

for split in folds_df.columns:
    # Create directories
    split_dir = save_path / split
    split_dir.mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "train" / "labels").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "images").mkdir(parents=True, exist_ok=True)
    (split_dir / "val" / "labels").mkdir(parents=True, exist_ok=True)

    # Create dataset YAML files
    dataset_yaml = split_dir / f"{split}_dataset.yaml"
    ds_yamls.append(dataset_yaml)

    with open(dataset_yaml, "w") as ds_y:
        yaml.safe_dump(
            {
                "path": split_dir.as_posix(),
                "train": "train",
                "val": "val",
                "names": classes,
            },
            ds_y,
        )

In [57]:
import shutil

for image, label in zip(images, labels):
    for split, k_split in folds_df.loc[image.stem].items():
        # Destination directory
        img_to_path = save_path / split / k_split / "images"
        lbl_to_path = save_path / split / k_split / "labels"

        # Copy image and label files to new directory (SamefileError if file already exists)
        shutil.copy(image, img_to_path / image.name)
        shutil.copy(label, lbl_to_path / label.name)

Train

In [63]:
from ultralytics import YOLO

weights_path = "./yolo11n.pt"
model = YOLO(weights_path, task="detect")

In [64]:
results = {}

# Define your additional arguments here
batch = 16
project = "kfold_demo"
epochs = 2

for k in range(ksplit):
    dataset_yaml = ds_yamls[k]
    model = YOLO(weights_path, task="detect")
    model.train(data=dataset_yaml, epochs=epochs, batch=batch, project=project)  # include any train arguments
    results[k] = model.metrics  # save output metrics for further analysis
    

Ultralytics 8.3.39 🚀 Python-3.9.6 torch-2.5.1 CPU (Apple M1 Pro)
engine/trainer: task=detect, mode=train, model=./yolo11n.pt, data=Deep-learning-12/train/2024-11-29_5-Fold_Cross-val/split_1/split_1_dataset.yaml, epochs=2, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=kfold_demo, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

train: Scanning /Users/NichlasKondrup/Desktop/Deep Learning/2/Deep_learning_project/datasets/Deep-learning-12/train/2024-11-29_5-Fold_Cross-val/split_1/train/labels... 448 images, 0 backgrounds, 0 corrupt: 100%|██████████| 448/448 [00:00<00:00, 979.28it/s] 

train: New cache created: /Users/NichlasKondrup/Desktop/Deep Learning/2/Deep_learning_project/datasets/Deep-learning-12/train/2024-11-29_5-Fold_Cross-val/split_1/train/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 1, len(boxes) = 654. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning /Users/NichlasKondrup/Desktop/Deep Learning/2/Deep_learning_project/datasets/Deep-learning-12/train/2024-11-29_5-Fold_Cross-val/split_1/val/labels... 113 images, 0 backgrounds, 0 corrupt: 100%|██████████| 113/113 [00:00<00:00, 2429.26it/s]

val: New cache created: /Users/NichlasKondrup/Desktop/Deep Learning/2/Deep_learning_project/datasets/Deep-learning-12/train/2024-11-29_5-Fold_Cross-val/split_1/val/labels.cache


Plotting labels to kfold_demo/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000417, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to kfold_demo/train4
Starting training for 2 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/2         0G      1.072       4.11      1.513         45        640:  18%|█▊        | 5/28 [00:49<03:47,  9.89s/it]


KeyboardInterrupt: 

## Train YOLOv11 with dataset

In [1]:
from ultralytics import YOLO
import torch

# Create a new YOLO model from scratch
model = YOLO("yolo11n.yaml")

# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolo11n.pt")

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

model = model.to(device)
# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data="Deep-Learning-12/data.yaml", epochs=10)

# Evaluate the model's performance on the validation set
results = model.val()

# Perform object detection on an image using the model
#results = model("https://ultralytics.com/images/bus.jpg")


Matplotlib is building the font cache; this may take a moment.
/Users/NichlasKondrup/Desktop/Deep Learning/2/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/NichlasKondrup/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=Deep-Learning-12/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source

RuntimeError: Dataset 'Deep-Learning-12/data.yaml' error ❌ 
Dataset 'Deep-Learning-12/data.yaml' images not found ⚠️, missing path '/Users/NichlasKondrup/Desktop/Deep Learning/2/Deep_learning_project/datasets/Deep-Learning-12/valid/images'
Note dataset download directory is '/Users/NichlasKondrup/Desktop/Deep Learning/2/Deep_learning_project/datasets'. You can update this in '/Users/NichlasKondrup/Library/Application Support/Ultralytics/settings.json'

In [29]:
results = model.predict(source="data/images",conf=0.2)



image 1/1 c:\Users\Bruger\OneDrive\Dokumenter\PyJects\Project\yolov5\data\images\lettuce_test.jpg: 512x640 1 hakket oksekod, 1 salat, 26.0ms
Speed: 0.0ms preprocess, 26.0ms inference, 2.0ms postprocess per image at shape (1, 3, 512, 640)


In [35]:
# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    print(boxes)
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([ 7., 17.], device='cuda:0')
conf: tensor([0.32103, 0.27564], device='cuda:0')
data: tensor([[4.19142e+01, 4.11390e+00, 6.25132e+02, 5.31442e+02, 3.21026e-01, 7.00000e+00],
        [2.90360e+01, 3.79087e+00, 6.29693e+02, 5.30580e+02, 2.75636e-01, 1.70000e+01]], device='cuda:0')
id: None
is_track: False
orig_shape: (538, 674)
shape: torch.Size([2, 6])
xywh: tensor([[333.52310, 267.77808, 583.21783, 527.32837],
        [329.36441, 267.18546, 600.65680, 526.78912]], device='cuda:0')
xywhn: tensor([[0.49484, 0.49773, 0.86531, 0.98016],
        [0.48867, 0.49663, 0.89118, 0.97916]], device='cuda:0')
xyxy: tensor([[ 41.91417,   4.11390, 625.13202, 531.44226],
        [ 29.03601,   3.79087, 629.69281, 530.58002]], device='cuda:0')
xyxyn: tensor([[0.06219, 0.00765, 0.92750, 0.98781],
        [0.04308, 0.00705, 0.93426, 0.98621]], device='cuda:0')


In [31]:
print(probs)

None
